# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Cassandra Database Modelling

The **event_datafile_new.csv** contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance on the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

### Create Keyspace

In [8]:
# Create a Keyspace 
query = "CREATE  KEYSPACE IF NOT EXISTS udacity \
   WITH REPLICATION = { \
      'class' : 'SimpleStrategy', 'replication_factor' : 1 \
   };"
try:
    session.execute(query);
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace("udacity")

**Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.**

- `SELECT artist, song_title, song_length FROM table1 WHERE session_id = 338 AND item_in_session = 4;`


- `SELECT artist, song, user FROM table2 WHERE user_id = 10 and session_id = 182;`
    

- `SELECT user FROM table3 WHERE song = 'All Hands Against His Own';`




### Query 1

We require the columns `artist`, `song_title`, `song_length` based on specific `session_id` and `item_in_session`. Based on this, we need to partition the table in a way that it looks for rows only for `session_id = 338` and `item_in_session = 4`. This allows us to filter-through unwanted rows and make the read of the table efficient.

We will set partition keys to both `session_id` and `item_in_session` as both of them will be used in the `WHERE` statement to get the desired rows.

In [35]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

try:
    session.execute("CREATE TABLE IF NOT EXISTS table1 (session_id int, item_in_session int, artist varchar, song_title varchar, song_length float,  PRIMARY KEY(session_id, item_in_session))");
except Exception as e:
    print(e)              

In [38]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## inserting data into table1
        query = "INSERT INTO table1(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [41]:
## select statement to verify the model is correct
query = "SELECT artist, song_title, song_length FROM table1 WHERE session_id = 338 AND item_in_session = 4;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
        
for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### Query 2

We require the columns `artist`, `song_title`, `user` based on specific `user_id` and `session_id`. Based on this, we need to partition the table in a way that it looks for rows only for `user_id = 10` and `session_id = 182`. This allows us to filter-through unwanted rows and make the read of the table efficient. We also need to sort the rows by `item_in_session` (ascending as it is not explicitly mentioned to sort descending), to do this we need to add `item_in_session` in the composite partition key.

We will set partition keys to both `user_id` and `session_id` as both of them will be used in the `WHERE` statement to get the desired rows. Along with the partition key, we also need a cluster key to sort the rows based on `item_in_session`, which will be the column itself.

One additional thing to take care of in this query is the column `user`. This will be a calculated column as we required both first and last name of a user together in this column.

In [53]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

try:
    session.execute("CREATE TABLE IF NOT EXISTS table2 (user_id int, session_id int, item_in_session int, artist varchar, song varchar, user varchar, PRIMARY KEY((user_id, session_id), item_in_session));")
except Exception as e:
    print(e)

In [54]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## insert data to table2
        query = "INSERT INTO table2(user_id, session_id, item_in_session, artist, song, user)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], (line[1] + " " + line[4])))

In [55]:
#Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "SELECT artist, song, user FROM table2 WHERE user_id = 10 and session_id = 182;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
        
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", user='Sylvie Cruz')
Row(artist='Three Drives', song='Greece 2000', user='Sylvie Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', user='Sylvie Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user='Sylvie Cruz')


### Query 3

This is a relatively simpler query. We only need to return a single column `user`, again, this will be a calculated column similar to what we did in Query 2. We will also add the column `song` in the table because we need to filter for the song **All Hands Against His Own**. `song` and `user` both will be partition keys.

One thing to keep in mind is the order of the column we add to the partition key - if we add `user` followed by `song`, the query will look at every user and then partition using the `song`. This is not efficient for a Cassandra database because it is not optimised for reads, and won't be able to process large amounts of data efficiently. In fact, the query won't work unless we add `ALLOW FILTERING` to the query. 

In [77]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

try:
    session.execute("CREATE TABLE IF NOT EXISTS table3 (user varchar, song varchar, PRIMARY KEY(song, user));")
except Exception as e:
    print(e)                    

In [78]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO table3(user, song)"
        query = query + "VALUES(%s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, ((line[1] + " " + line[4]), line[9]))

In [79]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "SELECT user FROM table3 WHERE song = 'All Hands Against His Own';"

try:
    rows = session.execute(query)
    
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(user='Jacqueline Lynch')
Row(user='Sara Johnson')
Row(user='Tegan Levine')


### Clearing Database

In [80]:
## TO-DO: Drop the table before closing out the sessions

session.execute("DROP TABLE IF EXISTS table1;")
session.execute("DROP TABLE IF EXISTS table2;")
session.execute("DROP TABLE IF EXISTS table3;")

### Connection Close

In [81]:
session.shutdown()
cluster.shutdown()